In [451]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
import warnings
import numpy as np
import requests
from io import StringIO
import nfl_data_py as nfl

# Suppress specific FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [452]:
# Define the base path and URLs as per your config file
BASE_CDN_PATH = "https://bigdatabowl2023.nyc3.cdn.digitaloceanspaces.com"
TRACKING_DATA_URL_TEMPLATE = BASE_CDN_PATH + "/raw/tracking_data/tracking_week_{week}.csv"
PLAYS_URL = BASE_CDN_PATH + "/raw/plays.csv"
GAMES_URL = BASE_CDN_PATH + "/raw/games.csv"
PLAYERS_URL = BASE_CDN_PATH + "/raw/players.csv"
COLORS_URL = BASE_CDN_PATH + "/raw/colors.csv"
TACKLES_URL = BASE_CDN_PATH + "/raw/tackles.csv"

# Replace 'week' with the actual week number you want to download
week_number = 1
TRACKING_DATA_URL = TRACKING_DATA_URL_TEMPLATE.format(week=week_number)

# Define a function to download and return a DataFrame
def download_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Assuming the data is in CSV format
        return pd.read_csv(StringIO(response.text))
    else:
        print(f"Failed to download data from {url}")
        return None


# Download the data
tracking_data = download_data(TRACKING_DATA_URL)
plays_data = download_data(PLAYS_URL)
players_data = download_data(PLAYERS_URL)
colors_data = download_data(COLORS_URL)
tackles_data = download_data(TACKLES_URL)
games_data = download_data(GAMES_URL) 

In [453]:
print(tracking_data.info())
print(plays_data.info())
print(players_data.info())
print(colors_data.info())
print(tackles_data.info())
print(games_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407439 entries, 0 to 1407438
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   gameId         1407439 non-null  int64  
 1   playId         1407439 non-null  int64  
 2   nflId          1346246 non-null  float64
 3   displayName    1407439 non-null  object 
 4   frameId        1407439 non-null  int64  
 5   time           1407439 non-null  object 
 6   jerseyNumber   1346246 non-null  float64
 7   club           1407439 non-null  object 
 8   playDirection  1407439 non-null  object 
 9   x              1407439 non-null  float64
 10  y              1407439 non-null  float64
 11  s              1407439 non-null  float64
 12  a              1407439 non-null  float64
 13  dis            1407439 non-null  float64
 14  o              1346397 non-null  float64
 15  dir            1346397 non-null  float64
 16  event          130268 non-null   object 
dtypes: float

In [454]:
df = pd.read_csv('/Users/nick/nfl-big-data-bowl-2024/data/final_pso_pipeline_results_week_1_completions_only_run2.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Loop through weeks 2 to 9
for week in range(1, 10):
    # Construct file path for the current week
    file_path = f'/Users/nick/nfl-big-data-bowl-2024/data/final_pso_pipeline_results_week_{week}_completions_only_run2.csv'
    
    # Read the data
    week_df = pd.read_csv(file_path)
    
    # Remove any unnamed columns
    week_df = week_df.loc[:, ~week_df.columns.str.contains('^Unnamed')]
    
    # Append to the original DataFrame
    df = pd.concat([df, week_df], ignore_index=True)


In [455]:
df.columns

Index(['nflId', 'frechet_distance', 'play_id', 'game_id', 'week_num',
       'displayName', 'position', 'gameId', 'playId', 'passResult',
       'expectedPointsAdded', 'defensiveTeam'],
      dtype='object')

In [456]:
#sanity check
plays_per_game = df.groupby('game_id')['play_id'].nunique()

# Calculating the average number of plays per game
average_plays_per_game = plays_per_game.mean()
average_plays_per_game

41.4921875

In [457]:
# Merge the DataFrames on 'playId' and 'nflId'
merged_df = pd.merge(df, tackles_data, left_on=['playId', 'nflId','gameId'], right_on=['playId', 'nflId','gameId'], how='left')

In [458]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29575 entries, 0 to 29574
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nflId                29575 non-null  float64
 1   frechet_distance     29575 non-null  float64
 2   play_id              29575 non-null  int64  
 3   game_id              29575 non-null  int64  
 4   week_num             29575 non-null  int64  
 5   displayName          29575 non-null  object 
 6   position             29575 non-null  object 
 7   gameId               29575 non-null  int64  
 8   playId               29575 non-null  int64  
 9   passResult           29458 non-null  object 
 10  expectedPointsAdded  29575 non-null  float64
 11  defensiveTeam        29575 non-null  object 
 12  tackle               4811 non-null   float64
 13  assist               4811 non-null   float64
 14  forcedFumble         4811 non-null   float64
 15  pff_missedTackle     4811 non-null  

In [459]:
# Replace NaN in 'passResult' with 'R' (for run plays)
df['passResult'].fillna('R', inplace=True)

In [460]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29575 entries, 0 to 29574
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nflId                29575 non-null  float64
 1   frechet_distance     29575 non-null  float64
 2   play_id              29575 non-null  int64  
 3   game_id              29575 non-null  int64  
 4   week_num             29575 non-null  int64  
 5   displayName          29575 non-null  object 
 6   position             29575 non-null  object 
 7   gameId               29575 non-null  int64  
 8   playId               29575 non-null  int64  
 9   passResult           29575 non-null  object 
 10  expectedPointsAdded  29575 non-null  float64
 11  defensiveTeam        29575 non-null  object 
dtypes: float64(3), int64(5), object(4)
memory usage: 2.7+ MB


High correlation here because the players are traviling farther and farther away from the frechet distance when a secondary player makes a tackle. 

In [461]:
# Summing Fréchet distance for each play for each team 
play_level_frechet_agg_team = df.groupby(['gameId', 'playId', 'defensiveTeam',]).agg(
    avg_frechet_per_play_per_player=('frechet_distance', 'mean'),
    players_in_swarm =('nflId', 'nunique')
).reset_index()

# Summing Fréchet distance for each play for each player
play_level_frechet_agg_player = df.groupby(['gameId', 'playId', 'nflId','displayName']).agg(
    avg_frechet_per_play_per_player=('frechet_distance', 'mean'),
).reset_index()



In [462]:
play_level_frechet_agg_team

,gameId,playId,defensiveTeam,avg_frechet_per_play_per_player,players_in_swarm
0,2022090800,56,LA,3.097332,5
1,2022090800,122,LA,8.087729,5
2,2022090800,167,LA,4.521092,5
3,2022090800,212,LA,2.835197,5
4,2022090800,236,LA,9.946312,4
...,...,...,...,...,...
5306,2022110700,3323,BAL,7.888510,5
5307,2022110700,3347,BAL,8.136728,5
5308,2022110700,3401,BAL,6.221733,5
5309,2022110700,3429,BAL,19.975470,5


In [463]:
# Aggregate at the team level to get averages across all plays per team
team_level_agg = play_level_frechet_agg_team.groupby('defensiveTeam').agg(
    avg_avg_frechet_per_play_per_player=('avg_frechet_per_play_per_player', 'mean'),
).reset_index()


In [464]:
play_level_frechet_agg_team

,gameId,playId,defensiveTeam,avg_frechet_per_play_per_player,players_in_swarm
0,2022090800,56,LA,3.097332,5
1,2022090800,122,LA,8.087729,5
2,2022090800,167,LA,4.521092,5
3,2022090800,212,LA,2.835197,5
4,2022090800,236,LA,9.946312,4
...,...,...,...,...,...
5306,2022110700,3323,BAL,7.888510,5
5307,2022110700,3347,BAL,8.136728,5
5308,2022110700,3401,BAL,6.221733,5
5309,2022110700,3429,BAL,19.975470,5


In [465]:
# Aggregate at the player level to get averages across all plaue
player_level_agg = play_level_frechet_agg_player.groupby(['nflId', 'displayName']).agg(
    avg_frechet_per_play=('avg_frechet_per_play_per_player', 'mean')
).reset_index()


In [466]:
play_level_frechet_agg_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5311 entries, 0 to 5310
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   gameId                           5311 non-null   int64  
 1   playId                           5311 non-null   int64  
 2   defensiveTeam                    5311 non-null   object 
 3   avg_frechet_per_play_per_player  5311 non-null   float64
 4   players_in_swarm                 5311 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 207.6+ KB


In [467]:
playfrechet_team_df = play_level_frechet_agg_team.merge(plays_data[['gameId', 'playId', 'yardsToGo', 'prePenaltyPlayResult','preSnapVisitorScore','preSnapHomeScore','gameClock']], on=['gameId', 'playId'], how='left')


In [468]:
playfrechet_team_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5311 entries, 0 to 5310
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   gameId                           5311 non-null   int64  
 1   playId                           5311 non-null   int64  
 2   defensiveTeam                    5311 non-null   object 
 3   avg_frechet_per_play_per_player  5311 non-null   float64
 4   players_in_swarm                 5311 non-null   int64  
 5   yardsToGo                        5311 non-null   int64  
 6   prePenaltyPlayResult             5311 non-null   int64  
 7   preSnapVisitorScore              5311 non-null   int64  
 8   preSnapHomeScore                 5311 non-null   int64  
 9   gameClock                        5311 non-null   object 
dtypes: float64(1), int64(7), object(2)
memory usage: 415.0+ KB


In [469]:
# Merge the aggregated data with the team colors data
playfrechet_team_df_with_colors = playfrechet_team_df.merge(colors_data, left_on='defensiveTeam', right_on='team_abbr')


In [470]:
playfrechet_team_df_with_colors.columns

Index(['gameId', 'playId', 'defensiveTeam', 'avg_frechet_per_play_per_player',
       'players_in_swarm', 'yardsToGo', 'prePenaltyPlayResult',
       'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock', 'team_abbr',
       'team_name', 'team_id', 'team_nick', 'team_conf', 'team_division',
       'team_color', 'team_color2', 'team_color3', 'team_color4',
       'team_logo_wikipedia', 'team_logo_espn', 'team_wordmark',
       'team_conference_logo', 'team_league_logo'],
      dtype='object')

In [471]:
# Function to convert game clock to a numerical value (minutes)
def convert_game_clock(clock_str):
    minutes, seconds = map(int, clock_str.split(':'))
    return minutes + seconds / 60.0

# Apply the conversion function to the gameClock column
playfrechet_team_df_with_colors['gameClockMinutes'] = playfrechet_team_df_with_colors['gameClock'].apply(convert_game_clock)


In [472]:
# Define the years and columns of interest
years = [2022]
columns = ['old_game_id', 'play_id', 'yards_after_catch', 
           'xyac_epa', 'xyac_mean_yardage', 'xyac_median_yardage', 
           'comp_yac_epa', 'comp_yac_wpa', 'week']

# Import the play-by-play data
pbp_data = nfl.import_pbp_data(years, columns)

# Filter for weeks 1-9
filtered_pbp_data = pbp_data[pbp_data['week'].between(1, 10)]

# Ensure the data types for merging columns are consistent
filtered_pbp_data['old_game_id'] = filtered_pbp_data['old_game_id'].astype(int)
playfrechet_team_df_with_colors['gameId'] = playfrechet_team_df_with_colors['gameId'].astype(int)

# Import the play_level_frechet data
# You would need to specify how to import this data, as the method is not provided in your query

# Merge the data
data = pd.merge(filtered_pbp_data, playfrechet_team_df_with_colors, left_on=['old_game_id', 'play_id'], right_on=['gameId', 'playId'])


2022 done.
Downcasting floats.


/var/folders/kj/29dyy8nn54v_615jgbd59t7h0000gn/T/ipykernel_9756/1368900374.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_pbp_data['old_game_id'] = filtered_pbp_data['old_game_id'].astype(int)


In [473]:
# Define the years and columns of interest
years = [2022]
columns = ['nflverse_play_id', 'is_screen_pass', 
           'is_play_action', 'is_contested_ball', 'week']


# Import the FTN data
ftn_data = nfl.import_ftn_data(years, columns)
ftn_data.columns
# Filter for weeks 1-9
filtered_ftn_data = ftn_data[ftn_data['week'].between(1, 9)]

# Ensure the data types for merging columns are consistent
filtered_ftn_data['week'] = filtered_ftn_data['week'].astype(int)
filtered_ftn_data['nflverse_play_id'] = filtered_ftn_data['nflverse_play_id'].astype(int)


data['week'] = data['week'].astype(int)
data['play_id'] = data['play_id'].astype(int)

# Merge the data on 'week' and 'nflverse_play_id' with a left join
merged_data = pd.merge(data, filtered_ftn_data,
                       left_on=['week', 'play_id'], 
                       right_on=['week', 'nflverse_play_id'],
                       how='left')



Downcasting floats.


/var/folders/kj/29dyy8nn54v_615jgbd59t7h0000gn/T/ipykernel_9756/790701669.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ftn_data['week'] = filtered_ftn_data['week'].astype(int)
/var/folders/kj/29dyy8nn54v_615jgbd59t7h0000gn/T/ipykernel_9756/790701669.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ftn_data['nflverse_play_id'] = filtered_ftn_data['nflverse_play_id'].astype(int)


In [474]:
merged_data.columns

Index(['old_game_id', 'play_id', 'yards_after_catch', 'xyac_epa',
       'xyac_mean_yardage', 'xyac_median_yardage', 'comp_yac_epa',
       'comp_yac_wpa', 'week', 'season', 'nflverse_game_id', 'possession_team',
       'offense_formation', 'offense_personnel', 'defenders_in_box',
       'defense_personnel', 'number_of_pass_rushers', 'players_on_play',
       'offense_players', 'defense_players', 'n_offense', 'n_defense',
       'ngs_air_yards', 'time_to_throw', 'was_pressure', 'route',
       'defense_man_zone_type', 'defense_coverage_type', 'gameId', 'playId',
       'defensiveTeam', 'avg_frechet_per_play_per_player', 'players_in_swarm',
       'yardsToGo', 'prePenaltyPlayResult', 'preSnapVisitorScore',
       'preSnapHomeScore', 'gameClock', 'team_abbr', 'team_name', 'team_id',
       'team_nick', 'team_conf', 'team_division', 'team_color', 'team_color2',
       'team_color3', 'team_color4', 'team_logo_wikipedia', 'team_logo_espn',
       'team_wordmark', 'team_conference_logo', 'te

In [475]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8037 entries, 0 to 8036
Data columns (total 58 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   old_game_id                      8037 non-null   int64  
 1   play_id                          8037 non-null   int64  
 2   yards_after_catch                7794 non-null   float32
 3   xyac_epa                         7794 non-null   float32
 4   xyac_mean_yardage                7794 non-null   float32
 5   xyac_median_yardage              7794 non-null   float32
 6   comp_yac_epa                     8033 non-null   float32
 7   comp_yac_wpa                     8033 non-null   float32
 8   week                             8037 non-null   int64  
 9   season                           8037 non-null   int64  
 10  nflverse_game_id                 8037 non-null   object 
 11  possession_team                  8037 non-null   object 
 12  offense_formation   

In [476]:
# Filter out plays with only one unique player
plays_more_than_one_player = merged_data['players_in_swarm'] > 1

# Filter the original DataFrame to include only these plays
filtered_completed_passes = merged_data[merged_data['play_id'].isin(plays_more_than_one_player.index)]

# Calculate average number of players per play
players_per_play = filtered_completed_passes.groupby(['play_id'])['players_in_swarm'].mean()

# Add a new column with the calculated average players per play
filtered_completed_passes['avg_players_in_swarm'] = players_per_play


In [477]:

# Step 2: Calculate the difference between yards_after_catch and xyac_mean_yardage
filtered_completed_passes['yac_diff_per_play'] =  filtered_completed_passes['xyac_mean_yardage'] - filtered_completed_passes['yards_after_catch']

In [478]:
filtered_completed_passes

,old_game_id,play_id,yards_after_catch,xyac_epa,xyac_mean_yardage,xyac_median_yardage,comp_yac_epa,comp_yac_wpa,week,season,...,team_wordmark,team_conference_logo,team_league_logo,gameClockMinutes,nflverse_play_id,is_screen_pass,is_play_action,is_contested_ball,avg_players_in_swarm,yac_diff_per_play
0,2022091107,202,8.0,1.480030,10.545964,9.0,1.384328,0.000774,1,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,13.700000,202,False,True,False,NaN,2.545964
1,2022091107,230,1.0,0.950097,4.795807,3.0,0.364364,0.002137,1,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,13.350000,230,False,True,False,NaN,3.795807
2,2022091107,301,6.0,0.953877,5.303057,4.0,1.143623,0.007203,1,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,11.316667,301,False,False,False,NaN,-0.696943
3,2022091107,301,6.0,0.953877,5.303057,4.0,1.143623,0.007203,1,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,11.316667,301,False,False,False,NaN,-0.696943
4,2022091107,412,0.0,0.604533,4.788820,3.0,0.000000,0.000000,1,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,9.183333,412,False,True,False,NaN,4.788820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8032,2022110610,4255,0.0,0.639721,4.481321,3.0,0.000000,0.000000,9,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,9.783333,4255,False,False,False,NaN,4.481321
8033,2022110610,4279,5.0,0.185568,3.068146,1.0,0.241729,0.085939,9,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,9.133333,4279,False,False,False,NaN,-1.931854
8034,2022110610,4348,1.0,0.739616,8.350624,5.0,0.021576,0.278875,9,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,7.066667,4348,False,False,False,NaN,7.350624
8035,2022110610,4383,4.0,0.500793,3.738529,2.0,0.678168,0.185790,9,2022,...,https://github.com/nflverse/nflfastR-data/raw/...,https://github.com/nflverse/nflfastR-data/raw/...,https://raw.githubusercontent.com/nflverse/nfl...,6.533333,4383,False,False,False,NaN,-0.261471


In [479]:
filtered_completed_passes.columns

Index(['old_game_id', 'play_id', 'yards_after_catch', 'xyac_epa',
       'xyac_mean_yardage', 'xyac_median_yardage', 'comp_yac_epa',
       'comp_yac_wpa', 'week', 'season', 'nflverse_game_id', 'possession_team',
       'offense_formation', 'offense_personnel', 'defenders_in_box',
       'defense_personnel', 'number_of_pass_rushers', 'players_on_play',
       'offense_players', 'defense_players', 'n_offense', 'n_defense',
       'ngs_air_yards', 'time_to_throw', 'was_pressure', 'route',
       'defense_man_zone_type', 'defense_coverage_type', 'gameId', 'playId',
       'defensiveTeam', 'avg_frechet_per_play_per_player', 'players_in_swarm',
       'yardsToGo', 'prePenaltyPlayResult', 'preSnapVisitorScore',
       'preSnapHomeScore', 'gameClock', 'team_abbr', 'team_name', 'team_id',
       'team_nick', 'team_conf', 'team_division', 'team_color', 'team_color2',
       'team_color3', 'team_color4', 'team_logo_wikipedia', 'team_logo_espn',
       'team_wordmark', 'team_conference_logo', 'te

In [480]:
# Save the DataFrame as a CSV file
filtered_completed_passes.to_csv('team_level_filtered_completed_passes.csv', index=False)


In [481]:
import pandas as pd

# Assuming completed_passes is your DataFrame
filtered_completed_passes['pasta_index'] = (filtered_completed_passes['yac_diff_per_play'] / filtered_completed_passes['avg_frechet_per_play_per_player'] )*100

# Calculate average PASTA score, YAC diff, and average Frechet distance for each team
team_averages = filtered_completed_passes.groupby(['defensiveTeam','team_wordmark','team_logo_espn']).agg({'pasta_index': 'mean', 'yac_diff_per_play': 'mean', 'avg_frechet_per_play_per_player': 'first','avg_players_in_swarm':'first'})


# Sort by PASTA score and reset the index to keep defensiveTeam as a column
team_pasta_scores = team_averages.sort_values(by='pasta_index', ascending=False).reset_index()

# Add PASTA rank
team_pasta_scores['PASTA RANK'] = team_pasta_scores['pasta_index'].rank(method='dense', ascending=False)

# Round all numerical columns to one decimal place
team_pasta_scores = team_pasta_scores.round(1)

# Optional: Drop the original index column if it's still there
if 'index' in team_pasta_scores.columns:
    team_pasta_scores.drop(columns=['index'], inplace=True)




In [482]:
team_pasta_scores.columns

Index(['defensiveTeam', 'team_wordmark', 'team_logo_espn', 'pasta_index',
       'yac_diff_per_play', 'avg_frechet_per_play_per_player',
       'avg_players_in_swarm', 'PASTA RANK'],
      dtype='object')

In [484]:
# Assuming team_pasta_scores is your DataFrame
columns_of_interest = ['team_wordmark', 'team_logo_espn','pasta_index', 'PASTA RANK', 'yac_diff_per_play', 'avg_frechet_per_play_per_player', 'avg_players_in_swarm']
team_pasta_scores = team_pasta_scores[columns_of_interest]


In [ ]:
from IPython.display import display, HTML

# Function to get the HTML representation of an image
def image_formatter(im):
    return f'<img src="{im}" width="100">'

# Create a copy of the DataFrame for display
display_df = team_pasta_scores.copy()

# Apply the image formatter
display_df['team_wordmark'] = display_df['team_wordmark'].map(image_formatter)

# Convert DataFrame to HTML and display
display(HTML(display_df.to_html(escape=False)))


In [ ]:
from scipy.stats import linregress

slope, intercept, r_value, p_value, std_err = linregress(team_pasta_scores['avg_frechet_per_play_per_player'], team_pasta_scores['yac_diff_per_play'])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import requests
from io import BytesIO
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

# Create the scatter plot with a regression line
fig, ax = plt.subplots(figsize=(15, 10))
sns.regplot(x='avg_frechet_per_play_per_player', y='yac_diff_per_play', data=team_pasta_scores, ax=ax, scatter=False, color='red')

# Invert X and Y axes
ax.invert_xaxis()  # This will show lower frechet distances on the right
ax.invert_yaxis()  # This will show lower yac differences on the top



# Plot each team with its logo
for index, row in team_pasta_scores.iterrows():
    # Load team logo from URL
    response = requests.get(row['team_logo_espn'])
    img = Image.open(BytesIO(response.content))

    # Convert image to a format suitable for matplotlib
    imagebox = OffsetImage(img, zoom=0.08)
    ab = AnnotationBbox(imagebox, (row['avg_frechet_per_play_per_player'], row['yac_diff_per_play']), frameon=False)
    ax.add_artist(ab)

# Set labels and title
ax.set_xlabel('Average Frechet Distance Per Player Per Play')
ax.set_ylabel('Average YAC Difference')
ax.set_title('Correlation between Average Frechet Distance Per Player Involved in Tackle Per Play and Average YAC Difference by Defensive Team')
ax.text(0.5, 0.95, f'R\u00b2 = {r_value**2:.2f}', ha='right', va='top', transform=ax.transAxes, color='red', fontsize=12)

plt.show()


Stopping here everything below this needs attention 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scatter plot for summed frechet_distance per player per play vs. yac_diff for each defensive team
plt.figure(figsize=(12, 8))
sns.scatterplot(data=team_agg_df, x='frechet_per_player', y='yac_diff', hue='defensiveTeam')
plt.title('Summed Frechet Distance Per Player Per Play vs. YAC Difference by Defensive Team')
plt.xlabel('Summed Frechet Distance Per Player Per Play')
plt.ylabel('YAC Difference')
plt.legend(title='Defensive Team', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
import pandas as pd

# Assuming your DataFrame is named completed_passes

# Group by game and play, then sum 'frechet_distance' and take 'yac_diff' for each play
aggregated_df = completed_passes.groupby(['game_id', 'play_id', 'defensiveTeam']).agg({'frechet_distance':'sum', 'yac_diff':'mean'}).reset_index()


In [ ]:
# Count the number of unique 'nflId' per play
player_count = completed_passes.groupby(['game_id', 'play_id'])['nflId'].nunique().reset_index(name='player_count')

# Merge this count with the aggregated_df
merged_df = pd.merge(aggregated_df, player_count, on=['game_id', 'play_id'], how='left')

# Calculate per-player frechet_distance statistics
merged_df['frechet_per_player'] = merged_df['frechet_distance'] / merged_df['player_count']


In [ ]:
merged_df.info()

In [ ]:
merged_df = merged_df.merge(colors_data, left_on = 'defensiveTeam', right_on= 'team_abbr', how='left')

In [ ]:
merged_df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scatter plot for frechet_distance per player vs. yac_diff
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='frechet_distance', y='yac_diff')
plt.title('Correlation between YAC Difference Per Play and Frechet Distance Per Play')
plt.xlabel('Frechet Distance Per Play')
plt.ylabel('YAC Difference Expected - Actual Per Play')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter the DataFrame for yac_diff below the threshold of 3
filtered_df = merged_df[merged_df['yac_diff'] > -20]

# Calculate the correlation
correlation = filtered_df['frechet_per_player'].corr(filtered_df['yac_diff'])

# Scatter plot for yac_diff vs. frechet_distance per player with the filtered data
plt.figure(figsize=(10, 6))
sns.scatterplot(data=filtered_df, x='yac_diff', y='frechet_per_player')

# Adding the correlation coefficient to the plot
plt.title(f'Correlation between Per-Player Involved in Tackle Frechet Distance and YAC Difference (YAC Diff < 3)\nCorrelation: {correlation:.2f}')
plt.xlabel('YAC Difference')
plt.ylabel('Frechet Distance Per Player Involved in Tackle')

plt.show()




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming 'merged_data' is your merged dataset




# Step 3: Assess the correlation
correlation = completed_passes[['yac_diff', 'play_frechet']].corr().iloc[0, 1]
print(f"Correlation between YAC difference and Frechet distance: {correlation:.3f}")

# Step 4: Create a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=completed_passes, x='yac_diff', y='play_frechet')
plt.title('Scatter Plot of YAC Difference vs. Frechet Distance')
plt.xlabel('Difference in YAC (Actual - Expected)')
plt.ylabel('Frechet Distance')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming 'merged_data' is your merged dataset

# Step 1: Filter for completed passes
completed_passes = merged_data[merged_data['passResult'] == 'C']

# Step 2: Calculate the difference between yards_after_catch and xyac_mean_yardage
completed_passes['yac_diff'] = completed_passes['yards_after_catch'] - completed_passes['xyac_mean_yardage']

# Step 3: Define a threshold for low unexpected YAC and filter
# Let's say we define "significantly lower" as at least 3 yards less than expected
threshold = 3
low_unexpected_yac = completed_passes[completed_passes['yac_diff'] <= threshold]

# Step 4: Assess the correlation in this subset
correlation = low_unexpected_yac[['yac_diff', 'play_frechet']].corr().iloc[0, 1]
print(f"Correlation in low unexpected YAC subset: {correlation:.3f}")

# Step 5: Create a scatter plot for this subset
plt.figure(figsize=(10, 6))
sns.scatterplot(data=low_unexpected_yac, x='yac_diff', y='play_frechet')
plt.title('Scatter Plot of YAC Difference vs. Frechet Distance in Low Unexpected YAC Subset')
plt.xlabel('Difference in YAC (Actual - Expected)')
plt.ylabel('Frechet Distance')
plt.show()


In [ ]:
merged_data.columns

In [ ]:
merged_data
# Step 2: Calculate the difference between yards_after_catch and xyac_mean_yardage
merged_data['yac_diff'] = merged_data['yards_after_catch'] - merged_data['xyac_mean_yardage']

In [ ]:
# Define the conditions
conditions = ['is_screen_pass', 'is_contested_ball', 'is_play_action', 'yac_diff']

# Initialize a dictionary to store the results
results = {}

for condition in conditions:
    if condition != 'yac_diff':
        # Filter based on the condition
        filtered_df = merged_data[merged_data[condition] == True]
    else:
        # Use the entire DataFrame for yac_diff condition
        filtered_df = merged_data

    # Ensure the DataFrame is not empty
    if not filtered_df.empty:
        # Sort and find the plays with the highest and lowest frechet distances
        highest_frechet = filtered_df.sort_values(by='play_frechet', ascending=False).head(1)
        lowest_frechet = filtered_df.sort_values(by='play_frechet', ascending=True).head(1)

        results[condition] = {
            'highest_frechet': highest_frechet[['week', 'game_id', 'play_id', 'play_frechet']],
            'lowest_frechet': lowest_frechet[['week', 'game_id', 'play_id', 'play_frechet']]
        }

        # For yac_diff condition, also find the highest and lowest yac_diff
        if condition == 'yac_diff':
            top_yac_diff = filtered_df.sort_values(by='yac_diff', ascending=False).head(1)
            lowest_yac_diff = filtered_df.sort_values(by='yac_diff', ascending=True).head(1)

            results[condition].update({
                'top_yac_diff': top_yac_diff[['week', 'game_id', 'play_id', 'yac_diff']],
                'lowest_yac_diff': lowest_yac_diff[['week', 'game_id', 'play_id', 'yac_diff']]
            })

# Print the results
for condition, data in results.items():
    print(f"Condition: {condition}")
    print("Highest Frechet:\n", data['highest_frechet'])
    print("Lowest Frechet:\n", data['lowest_frechet'])
    if condition == 'yac_diff':
        print("Top YAC Diff:\n", data['top_yac_diff'])
        print("Lowest YAC Diff:\n", data['lowest_yac_diff'])
    print("\n")

In [ ]:
test_df = pd.read_csv('/Users/nick/nfl-big-data-bowl-2024/data/specific_plays_paths.csv')

In [ ]:
# Assuming test_df is your DataFrame
print("Number of unique Play IDs:", merged_data['play_id'].nunique())
print("Number of unique Games:", merged_data['game_id'].nunique())


In [ ]:
# Group by 'playId' and count unique 'nflId's in each group
unique_nfl_ids_per_play = test_df.groupby('playId')['nflId'].nunique()

# Calculate the average number of unique 'nflId's per play
average_nfl_ids_per_play = unique_nfl_ids_per_play.mean()

print("Average number of unique NFL IDs per play:", average_nfl_ids_per_play)


In [ ]:
# Group by 'playId' and count unique 'frameId's in each group
unique_frames_per_play = test_df.groupby('playId')['frameId'].nunique()

# Calculate the average number of unique frames per play
average_frames_per_play = unique_frames_per_play.mean()

print("Average number of unique frames per play:", average_frames_per_play)



In [ ]:
test_df.info()